### Fed Z.1 Report

See: FRB Calendar: https://www.federalreserve.gov/data/releaseschedule.htm

FRBNY Consumer Credit Panel calendar: https://www.newyorkfed.org/microeconomics/calendar.html


In [1]:
%config Completer.use_jedi = False
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Change in liabilities

In [2]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=Z1&series=226131d24948eeb075a4766f667eccf6&lastobs=&' 
dt = 'from=01/01/1988&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'

d, df = clean_fed_data(base + srs + dt + oth)
df['Government'] = df[['LA214104005.Q', 'LA314104005.Q']].sum(axis=1)
df['NCB'] = df['LA144104005.Q'] - df['LA104104005.Q']
df.to_csv(data_dir / 'liab_z1_raw.csv', index_label='date')

In [3]:
# Data as a share of GDP
df = pd.read_csv(data_dir / 'liab_z1_raw.csv', index_col='date', 
                 parse_dates=True)
sh = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
                    )['A191RC'], axis=0)).dropna().loc['1989':] * 100
sh.loc['1989':].to_csv(data_dir / 'liabsh.csv', index_label='date')
liab_dt = dtxt(sh.index[-1])['qtr1']
write_txt(text_dir / 'liab_date.txt', liab_dt)

s = [('LA384104005.Q', 'blue!60!black', None, 0), 
     ('FL384190005.Q', 'cyan!88!blue', 'q', 0), 
     ('FL264090005.Q', 'red!90!magenta', None, 0)]
nodes = '\n'.join([end_node(sh[i], c, percent=True, date=d, size=1.2, offset=o) 
                   for i, c, d, o in s]).replace('\\\ \scriptsize Q', ' Q')
write_txt(text_dir / 'liabsh_node1.txt', nodes)

s = [('FL144190005.Q', 'green!72!black', 'q'), 
     ('FL154190005.Q', 'orange!90!red', None), 
     ('FL214190005.Q', 'cyan!60!white', None), 
     ('FL314190005.Q', 'blue!70!white', None)]
nodes = '\n'.join([end_node(sh[i], c, percent=True, date=d, size=1.2) 
                   for i, c, d in s]).replace('\\\ \scriptsize Q', ' Q')
write_txt(text_dir / 'liabsh_node.txt', nodes)
ss = s

s = [('LA144104005.Q', 'green!72!black', None), 
     ('LA154104005.Q', 'orange!90!red', None), 
     ('LA214104005.Q', 'cyan!60!white', None), 
     ('LA314104005.Q', 'blue!70!white', 'q')]
adj = node_adj(sh[[i[0] for i in s]])
nodes = '\n'.join([end_node(sh[i], c, percent=True, date=d, 
                            size=1.2, offset=adj[i]) 
                   for i, c, d in s]).replace('\\\ \scriptsize Q', ' Q')
write_txt(text_dir / 'liabsh_node2.txt', nodes)

# Calculate per capita adjusted for inflation
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']
pce = nipa_df(retrieve_table('T20304')['Data'], ['DPCERG']).loc[df.index,'DPCERG']
pr = (pce / pce.iloc[-1])
pc = df.divide(pr, axis=0).divide(pop, axis=0).dropna()
pc.loc['1989':].to_csv(data_dir / 'liabpc.csv', index_label='date')

s = [('LA384104005.Q', 'blue!60!black', None, 0), 
     ('FL384190005.Q', 'cyan!88!blue', 'q', -0.15), 
     ('FL264090005.Q', 'red!90!magenta', None, 0)]
nodes = '\n'.join([end_node(pc[i], c, date=d, size=1.2, offset=o, dollar=True) 
                   for i, c, d, o in s]).replace('\\\ \scriptsize Q', ' Q')
write_txt(text_dir / 'liabpc_node.txt', nodes)

In [4]:
# Text for overview of liabilities
srs = ['LA384104005.Q', 'FL384190005.Q', 'FL264090005.Q']
shlt = sh[srs].iloc[-1].apply('{:.1f} percent of GDP'.format)
pclt = pc[srs].iloc[-1].multiply(1000).apply('\${:,.0f} per capita'.format)
lvlt = df[srs].iloc[-1].divide(1_000_000).apply('\${:,.1f} trillion'.format)
ltdt = dtxt(sh.index[-1])['qtr1']
ltdt1 = dtxt(sh.index[-1])['qtr2']
debt_lvl = lvlt['LA384104005.Q']
debt_gdp = shlt['LA384104005.Q']
debt_pc = pclt['LA384104005.Q']
liab_lvl = lvlt['FL384190005.Q']
liab_gdp = shlt['FL384190005.Q']
liab_pc = pclt['FL384190005.Q']
for_lvl = lvlt['FL264090005.Q']
for_gdp = shlt['FL264090005.Q']
for_pc = pclt['FL264090005.Q']

text = ('The first and most-common approach to analyzing US liabilities '+
        'looks at \\textbf{debt}, which includes loans and debt securities, '+
        'such as bonds. In this approach, total debt is the sum of debt '+
        'held by each nonfinancial sector of the domestic economy: households, '+
        'businesses, and the government. The combined debt of US nonfinancial '+
        f'sectors is {debt_lvl} in {ltdt1}, equivalent to {debt_gdp}, or '+
        f'{debt_pc}. \n\nThe second approach is \\textbf{{total liabilities}}, '+
        'which include debt and all other financial obligations. Other '+
        'financial obligations include accounts payable, taxes payable, '+
        'pension obligations, intercompany debt, and other miscellaneous '+
        'liabilities. Total liabilities of nonfinancial sectors sum to '+
        f'{liab_lvl} in {ltdt}, or {liab_gdp}.\n\nThe last approach looks at '+
        '\\textbf{foreign financial claims} on the US. From a net US wealth '+
        'perspective, domestic liabilities held by domestic creditors cancel '+
        'out. It is therefore of interest to look at the portion of domestic '+
        'liabilities that are to foreign creditors. Foreign financial claims '+
        f'on the US total {for_lvl} in {ltdt}, equivalent to {for_gdp}, or '+
        f'{for_pc}. ')
write_txt(text_dir / 'liab_overview.txt', text)
print(text)

The first and most-common approach to analyzing US liabilities looks at \textbf{debt}, which includes loans and debt securities, such as bonds. In this approach, total debt is the sum of debt held by each nonfinancial sector of the domestic economy: households, businesses, and the government. The combined debt of US nonfinancial sectors is \$67.6 trillion in the second quarter of 2022, equivalent to 267.8 percent of GDP, or \$203,078 per capita. 

The second approach is \textbf{total liabilities}, which include debt and all other financial obligations. Other financial obligations include accounts payable, taxes payable, pension obligations, intercompany debt, and other miscellaneous liabilities. Total liabilities of nonfinancial sectors sum to \$91.2 trillion in 2022 Q2, or 361.0 percent of GDP.

The last approach looks at \textbf{foreign financial claims} on the US. From a net US wealth perspective, domestic liabilities held by domestic creditors cancel out. It is therefore of interes

In [5]:
n = {'FL384190005.Q': f'Total Liabilities {c_line("cyan!88!blue", see=False)}',
     'LA384104005.Q': f'\hspace{{2mm}}Debt {c_line("blue!60!black", see=False)}',
     'FL384122005.Q': '\hspace{4mm}Debt Securities',
     'FL384123005.Q': '\hspace{4mm}Loans',
     'FL264090005.Q': ('\hspace{2mm}Foreign Financial Claims '+
                       f'{c_line("red!90!magenta", see=False)}')}

result = sh[n.keys()]
lt = result.iloc[-2:].iloc[::-1]
pryr = result.iloc[-5].to_frame().T
pc = result.loc['2019-10-01'].to_frame().T
tbl = pd.concat([lt, pryr, pc]).T
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(tbl.columns)]

tbl.columns = cols
comp_years = ['2010', '1989']
for cy in comp_years:
    tbl[cy] = result.loc[cy].mean().rename(cy)
tbl = tbl.rename(n).applymap('{:.1f}'.format)
tbl.to_csv(data_dir / 'liabsh1.tex', sep='&', lineterminator='\\\ ', 
            quotechar=' ')

In [6]:
n = {'LA384104005.Q': 'Debt of Nonfinancial Sectors',
     'LA154104005.Q': ('\hspace{2mm}Households \& Nonprofits '+
                       f'{c_line("orange!90!red", see=False)}'),
     'FL153165005.Q': '\hspace{4mm}Home Mortgages',
     'FL153166000.Q': '\hspace{4mm}Consumer Credit',
     'LA144104005.Q': ('\hspace{2mm}Nonfinancial Businesses '+
                       f'{c_line("green!72!black", see=False)}'),
     'LA104104005.Q': '\hspace{4mm}Corporate',
     'FL104122005.Q': '\hspace{6mm}Debt Securities',
     'FL104123005.Q': '\hspace{6mm}Loans',
     'NCB': '\hspace{4mm}Noncorporate',
     'FL113165505.Q': '\hspace{6mm}Commercial Mortgages',
     'Government': '\hspace{2mm}Government',
     'LA214104005.Q': ('\hspace{4mm}State \& Local '+
                       f'{c_line("cyan!60!white", see=False)}'),
     'LA314104005.Q': ('\hspace{4mm}Federal '+
                       f'{c_line("blue!70!white", see=False)}')}

result = sh[n.keys()]
lt = result.iloc[-2:].iloc[::-1]
pryr = result.iloc[-5].to_frame().T
pc = result.loc['2019-10-01'].to_frame().T
tbl = pd.concat([lt, pryr, pc]).T
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(tbl.columns)]

tbl.columns = cols
comp_years = ['2010', '1989']
for cy in comp_years:
    tbl[cy] = result.loc[cy].mean().rename(cy)
tbl = tbl.rename(n).applymap('{:.1f}'.format)
tbl.to_csv(data_dir / 'liabsh2.tex', sep='&', lineterminator='\\\ ', 
            quotechar=' ')

In [7]:
clhh = c_line('orange!90!red')
clbus = c_line('green!72!black')
clfg = c_line('blue!70!white')
clslg = c_line('cyan!60!white')
ltdt = dtxt(sh.index[-1])['qtr1']
vals = df.divide(1_000_000).applymap('\${:,.1f} trillion'.format)
shtxt = sh.applymap('{:.1f} percent of GDP'.format)
hhval = vals['LA154104005.Q'].iloc[-1]
hhsh = shtxt['LA154104005.Q'].iloc[-1]
hhsh10 = f"{sh.loc['2010', 'LA154104005.Q'].mean():.1f} percent of GDP"
busval_liab = vals['FL144190005.Q'].iloc[-1]
busval = vals['LA144104005.Q'].iloc[-1]
bussh = shtxt['LA144104005.Q'].iloc[-1]
compare = compare_text(sh['LA144104005.Q'].iloc[-1], 
                       sh.loc['2019-10-01', 'LA144104005.Q'], 
                       [2.0, 10.0, 30.0])
busshpc = shtxt.loc['2019-10-01', 'LA154104005.Q'].replace(' of GDP', '')
nfcb_liab = vals['FL104190005.Q'].iloc[-1]
nfcb = vals['LA104104005.Q'].iloc[-1]
fgsh = shtxt['LA314104005.Q'].iloc[-1]
fgpc = shtxt.loc['2019-10-01', 'LA314104005.Q'].replace(' of GDP', '')
slgsh = shtxt['LA214104005.Q'].iloc[-1]
slgsh10 = f"{sh.loc['2010', 'LA214104005.Q'].mean():.1f} percent of GDP"
slgliab = shtxt['FL214190005.Q'].iloc[-1]
slgliab10 = f"{sh.loc['2010', 'FL214190005.Q'].mean():.1f} percent"

text = ('Liabilities vary \\textbf{by sector} and vary over '+
        'time within sectors. Households and nonprofits '+
        f'{clhh} have {hhval} in debt in {ltdt}, equivalent to '+
        f'{hhsh}. During the collapse of the housing bubble, in '+
        '2010, household and nonprofit debt was equivalent to '+
        f'{hhsh10}.\n\nIn {ltdt}, Private nonfinancial businesses '+
        f'{clbus}, corporate and noncorporate, have total '+
        f'liabilities of {busval_liab} and debt of {busval}. '+
        f'In {ltdt}, nonfinancial business debt is equivalent to '+
        f'{bussh}, {compare} the pre-COVID ratio of {busshpc}. '+
        f'Nonfinancial corporations have {nfcb_liab} in total '+
        f'liabilities and {nfcb} in debt. \n\n'+
        f'Federal government debt {clfg} is equivalent to {fgsh} '+
        f'in the latest data and {fgpc} in 2019 Q4. Federal '+
        'government debt has increased substantially since the '+
        f'great recession. State and local government debt {clslg} '+
        f'is equivalent to {slgsh} in {ltdt} and {slgsh10} in 2010. '+
        'Total liabilities for the sector, which include pensions, '+
        f'are {slgliab} in the latest data and {slgliab10} in 2010.')
write_txt(text_dir / 'liabsh.txt', text)
print(text)

Liabilities vary \textbf{by sector} and vary over time within sectors. Households and nonprofits (see {\color{orange!90!red}\textbf{---}}) have \$18.6 trillion in debt in 2022 Q2, equivalent to 73.5 percent of GDP. During the collapse of the housing bubble, in 2010, household and nonprofit debt was equivalent to 92.5 percent of GDP.

In 2022 Q2, Private nonfinancial businesses (see {\color{green!72!black}\textbf{---}}), corporate and noncorporate, have total liabilities of \$34.4 trillion and debt of \$19.5 trillion. In 2022 Q2, nonfinancial business debt is equivalent to 77.2 percent of GDP, slightly above the pre-COVID ratio of 74.1 percent. Nonfinancial corporations have \$24.7 trillion in total liabilities and \$12.6 trillion in debt. 

Federal government debt (see {\color{blue!70!white}\textbf{---}}) is equivalent to 104.2 percent of GDP in the latest data and 87.7 percent in 2019 Q4. Federal government debt has increased substantially since the great recession. State and local go

In [8]:
#sh.applymap(lambda x: value_text(x, 'eq', adj='gdp'))

### Real Debt Growth

In [9]:
# Real Debt Growth
n = {'FL384104005.Q': 'Total',
     'FL154104005.Q': '\hspace{-2mm}\cbox{orange!90!red}Households \& Nonprofits',
     'FL153165005.Q': '\hspace{4mm} Home Mortgages',
     'FL153166000.Q': '\hspace{4mm} Consumer Credit',
     'FL144104005.Q': '\hspace{-2mm}\cbox{green!72!black}Businesses',
     'FL104104005.Q': '\hspace{4mm}Corporate Businesses',
     'FL104122005.Q': '\hspace{6mm} Debt Securities',
     'FL104123005.Q': '\hspace{6mm} Loans',
     'NCB': '\hspace{4mm}Noncorporate Businesses',
     'FL113165505.Q': '\hspace{6mm} Commercial Mortgages',
     'FL214104005.Q': '\hspace{-2mm}\cbox{cyan!60!white}State \& Local Governments',
     'FL314104005.Q': '\hspace{-2mm}\cbox{blue!70!white}Federal Government'}
df = pd.read_csv(data_dir / 'liab_z1_raw.csv', index_col='date', 
                 parse_dates=True)
data = df.join(nipa_df(retrieve_table('T20304')['Data'], ['DPCERG'])['DPCERG'])
pr = data.DPCERG / data.DPCERG.iloc[-1]
real = data.divide(pr, axis=0).drop('DPCERG', axis=1)
res = growth_contrib_ann(real, 'FL384104005.Q').dropna()
res.to_csv(data_dir / 'liabgr.csv', index_label='date')
s = [('fed', 'blue!70!white', 'FL314104005.Q'), ('slg', 'cyan!60!white', 'FL214104005.Q'), 
     ('hh', 'orange!90!red', 'FL154104005.Q'), ('bus', 'green!72!black', 'FL144104005.Q')]
cbox = {name: c_box(color) for name, color, _ in s}
tot = value_text(res['FL384104005.Q'].iloc[-1], threshold=0.1, obj='plural')
stypes = [('contribution', '1'), ('contribution_to', '2')]
txt = {f'{name}{i}': value_text(res[col].iloc[-1], style, digits=1,
                                ptype='pp', threshold=0.1) 
       for style, i in stypes for name, _, col in s}
dt = dtxt(df.index[-1])
rn = {'FL144104005.Q': 'nonfinancial business debt', 
      'FL154104005.Q': 'household debt', 
      'FL214104005.Q': 'state and local government debt', 
      'FL314104005.Q': 'federal government debt'}
lt = res[rn.keys()].iloc[-1].rename(rn)
gct, _ = gc_desc(lt, 0.5, 2)

v30 = res[rn.keys()].iloc[-120:].mean().rename(rn)
gct30, _ = gc_desc(v30, 0.5, 2)
v3 = res[rn.keys()].iloc[-12:].mean().rename(rn)
gct3, _ = gc_desc(v3, 0.5, 2)
c1 = abs(v3).sort_values().index[-1]
ct1 = value_text(v3[c1], 'contribution', 'pp')
ctlt1 = value_text(v30[c1], 'contribution_of', 'pp', adj='average')
c2 = abs(v3).sort_values().index[-2]
ct2 = value_text(v3[c2], 'contribution', 'pp')
ctlt2 = value_text(v30[c2], 'contribution', 'pp')

t3 = value_text(v3.sum(), adj='average')
t30 = value_text(v30.sum(), 'increase_of', adj='average')
stdt = dtxt(res.index[-12])['qtr1']
txt1 = (f'Since {stdt}, real debt {t3}, compared to {t30} over the past 30 years.\n\n')

txt2 = (f'By sector, long-term real debt {gct30} Over the past '+
        f'three years, the {gct3} Since {stdt}, {c1} '+
        f'{ct1} to one-year real debt growth, on average, compared '+
        f'to {ctlt1} over the past 30 years. {c2[0].upper() + c2[1:]} '+
        f'{ct2} on average over the past three years, and {ctlt2} '+
        'on average over the past 30 years.\n\n')

txt3 = (f'Turning to the latest data, debt {tot} over the '+
        f'year ending {dt["qtr1"]}, after adjusting '+
        f'for inflation. In the latest quarter, the {gct} '+
        f'Federal government borrowing {cbox["fed"]} {txt["fed2"]} '+
        f'the overall change, while the state and local government '+
        f'{txt["slg1"]} {cbox["slg"]}. Households and nonprofits '+
        f'{txt["hh1"]} {cbox["hh"]}, and nonfinancial '+
        f'businesses {txt["bus1"]} {cbox["bus"]}.')
text = txt1 + txt2 + txt3
write_txt(text_dir / 'liabgr.txt', text)
print(text)

result = res[n.keys()]
tbl = result.iloc[-5:].iloc[::-1].T
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(tbl.columns)]

tbl.columns = cols
tbl['3-year'] = result.rolling(13).mean().iloc[-1].round(2)
tbl['10-year'] = result.rolling(41).mean().iloc[-1].round(2)
tbl['30-year'] = result.rolling(121).mean().iloc[-1].round(2)
tbl.index = tbl.index.map(n)
tbl = tbl.applymap('{:.2f}'.format)
tbl.to_csv(data_dir / 'liabgr.tex', sep='&', lineterminator='\\\ ', 
            quotechar=' ')

Since 2019 Q3, real debt increased at an average rate of 4.8 percent, compared to an average increase of 3.8 percent over the past 30 years.

By sector, long-term real debt growth is broad-based, with categories contributing relatively evenly. Over the past three years, the growth is relatively broad-based. The main contribution is an increase in federal government debt and nonfinancial business debt. Since 2019 Q3, federal government debt contributed 3.3 percentage points to one-year real debt growth, on average, compared to an average contribution of 1.4 percentage points over the past 30 years. Nonfinancial business debt contributed one percentage point on average over the past three years, and contributed 1.1 percentage points on average over the past 30 years.

Turning to the latest data, debt increased 0.2 percent over the year ending 2022 Q2, after adjusting for inflation. In the latest quarter, the low growth is the result of an increase in several categories that is partially 

### Wealth to GDP


In [10]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=Z1&series=2ce5701b67d5c62df54fe8fc8312f49c&lastobs=&'+
       'from=03/01/1988&to=12/31/2022&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, df = clean_fed_data(url)
df.to_csv(data_dir / 'uswealth_raw.csv', index_label='date')

In [11]:
df = pd.read_csv(data_dir / 'uswealth_raw.csv', index_col='date',
                 parse_dates=True)
d2 = {'FL264090005.Q': '\hspace{5mm} Less: ROW Claims on US',
      'FL882090265.Q': '\hspace{2mm} Net Claims on ROW',
      'FL892090005.Q': 'US Net Wealth',
      'LM112010005.Q': 'nfncb',
      'LM152010005.Q': '\hspace{2mm} Households \& Nonprofits',
      'LM212010095.Q': '\hspace{2mm} State \& Local Government',
      'LM315015005.Q': '\hspace{2mm} Federal Government',
      'LM662090003.Q': 'fncb',
      'LM882010405.Q': '\hspace{2mm} Domestic Corporations',
      'FL264194005.Q': '\hspace{5mm} US Claims on ROW',
      'LM155035005.Q': 'Real Estate', 
      'LM883164105.Q': 'Corporate Equities'}
data = df.rename(d2, axis=1)
data['\hspace{2mm} Noncorporate Businesses'] = data['nfncb'] + data['fncb']
data['Other'] = (data['US Net Wealth'] - data['Real Estate']
                 - data['Corporate Equities'])
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
ro = ['US Net Wealth', '\hspace{2mm} Households \& Nonprofits', 
      '\hspace{2mm} Noncorporate Businesses', 
      '\hspace{2mm} Domestic Corporations', '\hspace{2mm} Federal Government', 
      '\hspace{2mm} State \& Local Government', 
      '\hspace{2mm} Net Claims on ROW', '\hspace{5mm} US Claims on ROW', 
      '\hspace{5mm} Less: ROW Claims on US']
dsh = (data.div(gdp, axis=0) * 100).dropna()
res = dsh.loc[:,ro]

tbl = res.iloc[-3:].iloc[::-1].T
tbl[res.index[-5]] = res.iloc[-5]
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(tbl.columns)]

tbl.columns = cols
tbl['2019'] = res.loc['2019'].mean()
tbl['2005 --`07'] = res.loc['2005':'2007'].mean()
tbl['1989'] = res.loc['1989'].mean()
tbl = tbl.applymap('{:.1f}'.format)
tbl.to_csv(data_dir / 'uswealth.tex', sep='&', lineterminator='\\\ ', 
            quotechar=' ')

# Text
ltdt = dtxt(res.index[-1])['qtr1']
pop = nipa_df(retrieve_table('T20100')['Data'], ['B230RC'])['B230RC']
pc = (data['US Net Wealth'].divide(pop, axis=0)).dropna() * 1000
pcv = f'\${pc.iloc[-1].round(-2):,.0f}'
ltval = data['US Net Wealth'].iloc[-1] / 1_000_000
ltsh = res['US Net Wealth'].iloc[-1]

text = (f'US wealth totals \${ltval:.1f} trillion in {ltdt}, '+
        f'equivalent to {pcv} per capita, or a {ltsh / 100:.2f} '+
        f'multiple of GDP ({ltsh:.1f} percent of GDP.)')
write_txt(text_dir / 'uswealth.txt', text)
print(text)

keep = ['Corporate Equities', 'Real Estate', 'Other', 
        'US Net Wealth']
dft = dsh.loc['1989':, keep].divide(100)
dft.to_csv(data_dir / 'wealthgdp.csv', index_label='date')
tot = dft['US Net Wealth']
ltval = tot.iloc[-1]
t89 = tot.loc['1989'].mean()
cht = value_text((ltval - t89) * 100, ptype='pp')
ce = dft['Corporate Equities']
ce89 = ce.loc['1989'].mean()
ce99 = ce.loc['1999': '2000'].mean()
rr = dft['Real Estate']
rr89 = rr.loc['1989'].mean()
rr05 = rr.loc['2005': '2007'].mean()
ot = dft['Other']
ot89 = ot.loc['1989'].mean()

ceb = c_box('magenta!50!violet')
reb = c_box('green!80!blue')
otb = c_box('cyan!35!white')

txt = (f'The ratio of US wealth to GDP has {cht} since 1989, driven '+
       f'largely by increases in the market value of corporate '+
       'equities and residential real estate. The market value of '+
       f'corporate equities was equivalent to a {ce.iloc[-1]:.2f} '+
       f'multiple of GDP in {ltdt}, compared to {ce99:.2f} in 1999--'+
       f'2000, during the tech bubble, and to {ce89:.2f} in 1989 {ceb}. '+
       f'The market value of domestic residential real estate was '+
       f'equivalent to a {rr.iloc[-1]:.2f} multiple of GDP in {ltdt}, '+
       f'compared to {rr05:.2f} in 2005--2007, during the housing bubble, '+
       f'and {rr89:.2f} in 1989 {reb}.\n\nOn a net basis, all other US '+
       f'wealth is equivalent to a {ot.iloc[-1]:.2f} multiple of GDP '+
       f'in {ltdt} and a {ot89:.2f} multiple in 1989 {otb}. The other '+
       'category includes tangible assets of noncorporate businesses and '+
       'governments, and domestic financial claims on foreign assets. The '+
       'category also subtracts foreign financial claims on US assets, '+
       'for example foreign holdings of US corporate equities and '+
       'Treasury bonds.')
write_txt(text_dir / 'wealthgdp.txt', txt)
print(txt)

US wealth totals \$130.8 trillion in 2022 Q2, equivalent to \$392,900 per capita, or a 5.18 multiple of GDP (518.1 percent of GDP.)
The ratio of US wealth to GDP has increased 132.7 percentage points since 1989, driven largely by increases in the market value of corporate equities and residential real estate. The market value of corporate equities was equivalent to a 2.11 multiple of GDP in 2022 Q2, compared to 1.70 in 1999--2000, during the tech bubble, and to 0.60 in 1989 (see \cbox{magenta!50!violet}). The market value of domestic residential real estate was equivalent to a 1.80 multiple of GDP in 2022 Q2, compared to 1.85 in 2005--2007, during the housing bubble, and 1.34 in 1989 (see \cbox{green!80!blue}).

On a net basis, all other US wealth is equivalent to a 1.26 multiple of GDP in 2022 Q2 and a 1.91 multiple in 1989 (see \cbox{cyan!35!white}). The other category includes tangible assets of noncorporate businesses and governments, and domestic financial claims on foreign assets

### Business Debt

In [12]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=Z1&series=d720788442f3511d102b43eee2bddb41&lastobs=&'
dt = 'from=03/01/1988&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d = {'FL104122005.Q': 'Debt Securities',
     'FL143168005.Q': 'Bank Loans',
     'FL143165005.Q': 'Mortgages',
     'FL143169005.Q': 'Nonbank Loans',
     'FL144104005.Q': 'Total',
     'FL104104005.Q': 'Total Corporate',
     'FL114123005.Q': 'Total Noncorporate',
     'FL794122005.Q': 'Financial Debt Securities',
     'FL794123005.Q': 'Financial Loans',
     'FL794104005.Q': 'Financial Total',
     'FL423161705.Q': 'Agency MBS'}

r = requests.get(url)
df = pd.read_csv(io.StringIO(r.content.decode('utf-8')), skiprows=5, index_col=0)[d.keys()].rename(d, axis=1)
df.index = pd.to_datetime(df.index)
df['Bank Loans and Mortgages'] = df['Bank Loans'] + df['Mortgages']
df['Other'] = df['Financial Debt Securities'] - df['Agency MBS']

data = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
               )['A191RC'], axis=0)).dropna() * 100
data.loc['1989':].to_csv(data_dir / 'busdebtgdp2.csv', index_label='date', 
                         float_format='%g')

In [13]:
lt_date = f'{data.index[-1].year} Q{data.index[-1].quarter}'
total = df['Total'].iloc[-1] / 1_000
corp = df['Total Corporate'].iloc[-1] / 1_000
corp_sh = corp / total * 100
tot_gdp = data['Total'].iloc[-1]
tot_gdp_3 = data['Total'].iloc[-13]

date_3 = f'{data.index[-13].year} Q{data.index[-13].quarter}'
tot_3 = tot_gdp - tot_gdp_3
ds_3 = data['Debt Securities'].iloc[-1] - data['Debt Securities'].iloc[-13]
nb_3 = data['Nonbank Loans'].iloc[-1] - data['Nonbank Loans'].iloc[-13]

if tot_3 >= 1:
    tot_text = 'increased faster than'
    tot_text2 = 'increased'
elif (tot_3 < 1) & (tot_3 >= -1):
    tot_text = 'grown at about the same rate as'
    tot_text2 = 'grew'
else:
    tot_text = 'fallen relative to'
    tot_text2 = 'fell'
    
finmax = data['Financial Total'].max()
finmaxdt = f'{data["Financial Total"].idxmax().year} Q{data["Financial Total"].idxmax().quarter}'
finlt = data['Financial Total'].iloc[-1]

In [14]:
busdebt = (f'As of {lt_date}, \\textbf{{nonfinancial business debt}}--the debt '+
           'security and loan liabilities of nonfinancial businesses--both '+
           f'corporate and non-corporate--totals \${total:,.0f} billion, with '+
           f'\${corp:,.0f} billion ({corp_sh:,.1f} percent) held by corporate '+
           'businesses. Over the past three years, nonfinancial business debt '+
           f'has {tot_text} overall economic activity. As a share of GDP, '+
           f'nonfinancial business debt {tot_text2} by {tot_3:.1f} percentage '+
           f'points to {tot_gdp:.1f} percent in {lt_date} from {tot_gdp_3:.1f} '+
           f'percent in {date_3}. The vast majority of the increase, {nb_3:.1f} '+
           'percentage points, comes from nonbank loans (see\\cbox{{blue}}).')
write_txt(text_dir / 'busdebtgdp.txt', busdebt)

fintext = ('Domestic financial sector debt has fallen as a share of '+
           f'GDP to {finlt:.1f} percent in {lt_date} from a housing-bubble '+
           f'peak of {finmax:.1f} percent in {finmaxdt}.')
write_txt(text_dir / 'findebtgdp.txt', fintext)
print(busdebt, '\n')
print(fintext)

As of 2022 Q2, \textbf{nonfinancial business debt}--the debt security and loan liabilities of nonfinancial businesses--both corporate and non-corporate--totals \$19,457 billion, with \$12,560 billion (64.6 percent) held by corporate businesses. Over the past three years, nonfinancial business debt has increased faster than overall economic activity. As a share of GDP, nonfinancial business debt increased by 1.8 percentage points to 77.1 percent in 2022 Q2 from 75.3 percent in 2019 Q2. The vast majority of the increase, 2.0 percentage points, comes from nonbank loans (see\cbox{{blue}}). 

Domestic financial sector debt has fallen as a share of GDP to 74.9 percent in 2022 Q2 from a housing-bubble peak of 122.7 percent in 2008 Q4.


### Business Assets

In [15]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=Z1&series=eaf7407a6299f192955e236fed258708&lastobs=&'
dt = 'from=03/01/1988&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, df = clean_fed_data(url, dtype='full')
pd.to_pickle(d,data_dir / 'bus_assets.pickle')
df.columns = [c[:-2] for c in df.columns]
df['Bus_A'] = df[['FL112000005', 'FL102000005']].sum(axis=1)
df['Bus_FA'] = df[['FL114090005', 'FL104090005']].sum(axis=1)
df['Bus_NFA'] =  df[['LM112010005', 'LM102010005']].sum(axis=1)
df.to_csv(data_dir / 'bus_assets_raw.csv', index_label='date')

In [16]:
# Data as a share of GDP
df = pd.read_csv(data_dir / 'bus_assets_raw.csv', index_col='date', 
                 parse_dates=True)
sh = (df.div(nipa_df(retrieve_table('T10105')['Data'], ['A191RC']
                    )['A191RC'], axis=0)).dropna().loc['1989':] * 100
sh.loc['1989':].to_csv(data_dir / 'bus_asset_sh.csv', index_label='date')
#d = pd.read_pickle(data_dir / 'bus_assets.pickle')

In [17]:
# Nodes for latest values
s = {'LM112010005': 'noncorporate nonfinancial',
     'FL114090005': 'noncorporate financial',
     'LM102010005': 'corporate nonfinancial',
     'FL104090005': 'corporate financial'}
sdf = sh[s.keys()].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 15)).to_dict()
val = sdf.to_dict()
dtp = dtxt(sh.index[-1] + pd.DateOffset(months=2))['datetime']
nodes = [f'\\absnode{{{{{dtp}}}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}}}' 
         for i in s.keys()]
dtv = dtxt(sh.index[-1])['qtr1'].replace(' ', '\\\\ \scriptsize ')
dtn = f'\\absnode{{{{{dtp}}}}}{{{sdf.cumsum().iloc[-1] + 25.0}}}{{\scriptsize {dtv}:}}'
nodes.append(dtn)
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'bus_asset_nodes.txt', nodetext)

In [18]:
colors = {'LM112010005': 'orange!50!red!45!white',
          'FL114090005': 'orange!55!red',
          'LM102010005': 'green!52!blue!55!white',
          'FL104090005': 'green!70!yellow!70!lightgray!90!white'}
cb = {i: c_box(c) for i, c in colors.items()}
cb2 = {i: c_box(c, see=False) for i, c in colors.items()}

dft = df.divide(1_000_000).applymap('\${:.1f} trillion'.format)
sht = sh.applymap(lambda x: value_text(x, 'eq', adj='gdp'))
sht2 = sh.applymap(lambda x: value_text(x, 'eq'))
sht3 = sh.applymap(lambda x: value_text(x, 'plain'))
ltdt = dtxt(sh.index[-1])['qtr1']
ltdt2 = dtxt(sh.index[-1])['qtr2']

text = ('Combined \\textbf{assets of nonfinancial businesses} are valued '+
        f'at {dft.Bus_A.iloc[-1]} in {ltdt2}, {sht.Bus_A.iloc[-1]}. These '+
        'include financial assets and nonfinancial assets. Financial assets '+
        'include cash and deposits, equity in other businesses, trade '+
        f'receivables, and other financial assets, and total {dft.Bus_FA.iloc[-1]}. '+
        'Nonfinancial, or tangible, assets include real '+
        'estate, equipment, inventories, and intellectual property '+
        f'products, and total {dft.Bus_NFA.iloc[-1]}.\n\nNonfinancial '+
        f'corporations have assets valued at {dft.FL102000005.iloc[-1]}, or '+
        f'{sht3.FL102000005.iloc[-1]} of GDP. These include nonfinancial assets '+
        f'{cb["LM102010005"]} valued at {sht3.LM102010005.iloc[-1]} of GDP and '+
        f'financial assets {cb["FL104090005"]} valued at '+
        f'{sht3.FL104090005.iloc[-1]}, as of {ltdt}. Noncorporate '+
        f'business assets are valued at {dft.FL112000005.iloc[-1]}, '+
        f'{sht.FL112000005.iloc[-1]}. Tangible assets {cb["LM112010005"]} are '+
        f'{sht.LM112010005.iloc[-1]}, and include {dft.LM115035023.iloc[-1]} '+
        f'in rental housing. Financial assets for the sector {cb["FL114090005"]} '+
        f'are {sht.FL114090005.iloc[-1]}.')
write_txt(text_dir / 'bus_assets.txt', text)
print(text)

Combined \textbf{assets of nonfinancial businesses} are valued at \$80.7 trillion in the second quarter of 2022, equivalent to 319.6 percent of GDP. These include financial assets and nonfinancial assets. Financial assets include cash and deposits, equity in other businesses, trade receivables, and other financial assets, and total \$32.4 trillion. Nonfinancial, or tangible, assets include real estate, equipment, inventories, and intellectual property products, and total \$48.2 trillion.

Nonfinancial corporations have assets valued at \$54.8 trillion, or 217.0 percent of GDP. These include nonfinancial assets (see \cbox{green!52!blue!55!white}) valued at 115.9 percent of GDP and financial assets (see \cbox{green!70!yellow!70!lightgray!90!white}) valued at 101.0 percent, as of 2022 Q2. Noncorporate business assets are valued at \$25.9 trillion, equivalent to 102.6 percent of GDP. Tangible assets (see \cbox{orange!50!red!45!white}) are equivalent to 75.2 percent of GDP, and include \$10

In [19]:
n = {'FL102000005': 'Corporate Total',
     'LM102010005': f'\hspace{{2mm}}Nonfinancial Assets  {cb2["LM102010005"]}',
     'LM105035005': '\hspace{4mm}Real Estate',
     'LM105015205': '\hspace{4mm}Equipment',
     'LM105013765': '\hspace{4mm}IP Products',
     'LM105020005': '\hspace{4mm}Inventories',
     'FL104090005': f'\hspace{{2mm}}Financial Assets  {cb2["FL104090005"]}',
     'FL112000005': 'Noncorporate Total',
     'LM112010005': f'\hspace{{2mm}}Nonfinancial Assets  {cb2["LM112010005"]}',
     'LM115035005': '\hspace{4mm}Real Estate',
     'LM115035023': '\hspace{6mm}Residential',
     'LM115015205': '\hspace{4mm}Equipment',
     'LM115013765': '\hspace{4mm}IP Products',
     'LM115020005': '\hspace{4mm}Inventories',
     'FL114090005': f'\hspace{{2mm}}Financial Assets  {cb2["FL114090005"]}',}

result = sh[n.keys()]
lt = result.iloc[-3:].iloc[::-1]
pryr = result.iloc[-5].to_frame().T
pc = result.loc['2019-10-01'].to_frame().T
tbl = pd.concat([lt, pryr, pc]).T
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(tbl.columns)]

tbl.columns = cols
comp_years = ['2010', '1989']
for cy in comp_years:
    tbl[cy] = result.loc[cy].mean().rename(cy)
tbl = tbl.rename(n).applymap('{:.1f}'.format)
tbl.to_csv(data_dir / 'bus_asset.tex', sep='&', lineterminator='\\\ ', 
            quotechar=' ')

### Tobin's Q

In [20]:
# Tobin's Q
color = 'green!70!black'
s = df.loc['1989':, 'FL103164106']
s.name = 'value'
s.to_csv(data_dir / 'tobinq.csv', index_label='date')
write_txt(text_dir / 'tobinq_node.txt', 
          end_node(s, color, date='q'))

ltdt = dtxt(s.index[-1])['qtr1']
prdt = dtxt(s.index[-2])['qtr1']
pcval = s.loc['2019'].mean()
text = (f'As of {ltdt}, the ratio is {s[-1]:.1f} {c_line(color)}, '+
        f'following {s[-2]:.1f} in {prdt}, and compared to an average '+
        f'of {pcval:.1f} in 2019.')
write_txt(text_dir / 'tobinq.txt', text)
print(text)

As of 2022 Q2, the ratio is 159.9 (see {\color{green!70!black}\textbf{---}}), following 192.9 in 2022 Q1, and compared to an average of 165.1 in 2019.


### Household debt

In [21]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=Z1&series=21a69f49792f26a66791418647f75234&lastobs=&'
dt = 'from=03/01/1988&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d = {'FL153165105.Q': 'Mortgages',
     'FL153166000.Q': 'Consumer Credit',
     'FL154190005.Q': 'Total'}
r = requests.get(url)
df = (pd.read_csv(io.StringIO(r.content.decode('utf-8')), skiprows=5, 
                  index_col=0, parse_dates=True)[d.keys()]
        .rename(d, axis=1).divide(1000))
df['Other'] = df['Total'] - df['Consumer Credit'] - df['Mortgages']
table_store_fa = df.divide(1000)
dpi = nipa_df(retrieve_table('T20100')['Data'], ['A067RC'])['A067RC']
data = (df.div(dpi, axis=0)).dropna() * 100_000
data.loc['1989':].to_csv(data_dir / 'hhdebt.csv', index_label='date')
table_store_fa_dpi = data

In [22]:
date = dtxt(data.index[-1])['qtr1']
maxdate = dtxt(data.Total.idxmax())['qtr1']

totval = (df['Total'].iloc[-1] / 1000)
mortval = (df['Mortgages'].iloc[-1] / 1000)
mortsh = mortval / totval * 100
ccval = (df['Consumer Credit'].iloc[-1] / 1000)
ccsh = ccval / totval * 100
totrt = data['Total'].iloc[-1]
maxrt = data['Total'].max()
dpi3 = dpi.pct_change(12).iloc[-1] * 100
rt3 = df.Total.pct_change(12).iloc[-1] * 100
ch3 = data.Total.diff(12).iloc[-1]  
ch3txt = (value_text(ch3, style='increase_by', ptype='pp', threshold=0.3)
          .replace('decreased', 'fallen').replace('was', 'been'))
url = 'https://www.federalreserve.gov/releases/z1/current/default.htm'  
text1 = ('The liabilities of households and nonprofit institutions '+
         f'total \${totval:,.1f} trillion in {date}, as '+
         f'\href{{{url}}}{{reported}} by the Federal Reserve. Home mortgages are the '+
         f'main household liability, and total \${mortval:,.1f} trillion '+
         '(see\cbox{blue!60!violet}). Consumer credit liabilities include auto '+
         'loans, credit card debt, student loans, and other personal loans, and '+
         f'total \${ccval:,.1f} trillion (see\cbox{{magenta}}). The '+
         'remaining liabilities are primarily attributable to nonprofits '+
         '(see\cbox{orange!80!yellow}).')

text2 = ('The ratio of household and nonprofit debt to disposable personal '+
         f'income has fallen to {totrt:.1f} percent in {date} from its '+
         f'housing-bubble peak of {maxrt:.1f} percent in {maxdate}. Over '+
         'the past three years, household and nonprofit debt has '+
         f'{value_text(rt3, threshold=0.4)} while disposable personal '+
         f'income has {value_text(dpi3, threshold=0.4)}. As a result, the '+
         'ratio of household and nonprofit debt to disposable personal income '+
         f'has {ch3txt}.')
write_txt(text_dir / 'hhdebt1.txt', text1)    
write_txt(text_dir / 'hhdebt2.txt', text2)
print(text1, '\n\n', text2)

The liabilities of households and nonprofit institutions total \$18.9 trillion in 2022 Q2, as \href{https://www.federalreserve.gov/releases/z1/current/default.htm}{reported} by the Federal Reserve. Home mortgages are the main household liability, and total \$12.2 trillion (see\cbox{blue!60!violet}). Consumer credit liabilities include auto loans, credit card debt, student loans, and other personal loans, and total \$4.6 trillion (see\cbox{magenta}). The remaining liabilities are primarily attributable to nonprofits (see\cbox{orange!80!yellow}). 

 The ratio of household and nonprofit debt to disposable personal income has fallen to 102.7 percent in 2022 Q2 from its housing-bubble peak of 136.6 percent in 2007 Q4. Over the past three years, household and nonprofit debt has increased 17.0 percent while disposable personal income has increased 13.6 percent. As a result, the ratio of household and nonprofit debt to disposable personal income has increased by 3.4 percentage points.


### Consumer Credit

In [23]:
if os.path.exists(data_dir / 'ccp_hist.csv'):
    h = pd.read_csv(data_dir / 'ccp_hist.csv', index_col='date', 
                    parse_dates=True)
else:
    # Historical data
    print('Retrieving historical data')
    url = ('https://www.newyorkfed.org/medialibrary/media/research/'+
           'national_economy/householdcredit/pre2003_data.xlsx')
    h = pd.read_excel(url, sheet_name='Total Balance Data', skiprows=2, 
                      nrows=6, usecols='A:S', index_col=0).dropna(axis=1).T
    h.index = pd.date_range(start='1999-01-01', end='2002-10-01', freq='QS')
    h.to_csv(data_dir / 'ccp_hist.csv', index_label='date')
    
# Download latest data    
d1 = pd.to_datetime('today') - pd.DateOffset(weeks=7)
dt1 = f'{d1.year}q{d1.quarter}.xlsx'
d2 = pd.to_datetime('today') - pd.DateOffset(weeks=20)
dt2 = f'{d2.year}q{d2.quarter}.xlsx'

def retrieve_frbny_data(dt):
    url = ('https://www.newyorkfed.org/medialibrary/interactives/'+
           f'householdcredit/data/xls/hhd_c_report_')
    return pd.read_excel(url+dt, sheet_name='Page 3 Data', 
                         header=3, index_col=0)

try:
    df = retrieve_frbny_data(dt1)
except ValueError:
    df = retrieve_frbny_data(dt2)
    print(f'{dt1} Not available. Using {dt2} instead.')
df.index = [pd.to_datetime(f'20{x.replace(":", "-")}') for x in df.index]
df = df.dropna(axis=1)  
data = pd.concat([h, df])
data['Mortgage Total'] = data['Mortgage'] + data['HE Revolving']
data['Consumer Credit'] = data['Total'] - data['Mortgage Total']
data.to_csv(data_dir / 'hhccpdebt.csv', index_label='date')

In [24]:
data = pd.read_csv(data_dir / 'hhccpdebt.csv', index_col='date', 
                    parse_dates=True)
dpi = (nipa_df(retrieve_table('T20100')['Data'], ['A067RC'])
       .A067RC  / 1_000_000)
sh = (data.div(dpi, axis=0)).dropna(how='all') * 100
sh.loc['1999':].to_csv(data_dir / 'hhcdebt.csv', index_label='date')
ltdate = dtxt(sh.index[-1])['qtr1']
ltval = sh['Consumer Credit'].iloc[-1]
one_yr_diff = value_text(sh['Consumer Credit'].diff(4).iloc[-1], 
                         adj='total', ptype='pp')
cl = c_line('lime!55!green')
text = ('The latest comparable figure from the FRBNY data discussed in the '+
        f'previous section, which covers {ltdate}, shows consumer credit '+
        f'is equivalent to {ltval:.1f} percent of annual disposable '+
        f'personal income {cl}. Over the past year, the ratio {one_yr_diff}.')
write_txt(text_dir / 'cc_dpi_2.txt', text)
print(text)

st = end_node(sh['Student Loan'], 'green!80!blue', date='q', 
              full_year=True, offset=0.35, percent=True)
au = end_node(sh['Auto Loan'], 'blue!60!cyan', percent=True)
cc = end_node(sh['Credit Card'], 'red', percent=True)
nodes = '\n'.join([st, au, cc])
write_txt(text_dir / 'cc_nodes.txt', nodes)

c = [('Mortgage', 'gray'), ('HE Revolving', 'gray'), 
     ('Auto Loan', 'blue!60!cyan'), ('Credit Card', 'red'), 
     ('Student Loan', 'green!80!blue'), ('Total', 'gray')]
yr3ch = sh.diff(12).iloc[-1].sort_values(ascending=True)
d = dtxt(sh.index[-1])
d3 = dtxt(sh.index[-13])['qtr1']
cc_dates = d3 + '--' + d['qtr4']
write_txt(text_dir / 'cc_dates.txt', cc_dates)

bar = pd.DataFrame({'color': pd.Series({name: color for name, color in c}), 
                    'value': yr3ch}).dropna().sort_values('value')
bar.index.name = 'name'
bar.to_csv(data_dir / 'ccdebtbar.csv', index_label='name')

The latest comparable figure from the FRBNY data discussed in the previous section, which covers 2022 Q3, shows consumer credit is equivalent to 24.1 percent of annual disposable personal income (see {\color{lime!55!green}\textbf{---}}). Over the past year, the ratio increased by a total of 0.9 percentage point.


In [25]:
# Consumer credit text
totval = data['Total'].iloc[-1]
totval2 = sh['Total'].iloc[-1]
totvalch = data['Total'].diff(12).iloc[-1]
tvdir = value_text(totvalch, 'increase_by', ptype='trillion', 
                   dollar=True, threshold=0.1, digits=2)
dpich = dpi.diff(12).iloc[-1]
dpidir = value_text(dpich, 'increase_of', ptype='trillion', 
                   dollar=True, threshold=0.1, digits=2)    
totvalch2 = sh['Total'].diff(12).iloc[-1]
tvdir2 = value_text(totvalch2, ptype='pp', threshold=0.1)
url = 'https://www.newyorkfed.org/microeconomics/hhdc/background.html'
txt = (f'Federal Reserve Bank of New York (FRBNY) \href{{{url}}}{{analysis}} '+
       f'of Equifax data shows \\${totval:.1f} trillion in total consumer '+
       f'debt in {d["qtr2"]}, which is equivalent to {totval2:.1f} percent '+
       f'of disposable personal income. Over the past three years, total '+
       f'consumer debt has {tvdir}, compared to {dpidir} in disposable personal '+
       f'income. As a result, the ratio of total consumer debt to disposable '+
       f'personal income {tvdir2} over this period.')
write_txt(text_dir / 'hhcdebt3.txt', txt)
print(txt)

Federal Reserve Bank of New York (FRBNY) \href{https://www.newyorkfed.org/microeconomics/hhdc/background.html}{analysis} of Equifax data shows \$16.5 trillion in total consumer debt in the third quarter of 2022, which is equivalent to 88.3 percent of disposable personal income. Over the past three years, total consumer debt has increased by \$2.55 trillion, compared to an increase of \$2.24 trillion in disposable personal income. As a result, the ratio of total consumer debt to disposable personal income increased 3.5 percentage points over this period.


In [26]:
tot, shr, gr = {}, {}, {}
for series in ['Mortgage Total', 'Auto Loan', 'Student Loan', 'Credit Card']:
    tot[series] = data[series].iloc[-1] * 1000
    shr[series] = sh[series].iloc[-1]
    grtmp = (sh[series].diff(12)).iloc[-1]
    casual = True if series == 'Mortgage Total' else False
    style = 'increase_of' if series != 'Mortgage Total' else 'increase_by'
    gr[series] = value_text(grtmp, style, ptype='pp', 
                            threshold=0.1, casual=casual)
        
mgr = sh['Mortgage Total'].diff(12).iloc[-1]        
#gr['Mortgage Total'] = f'fell by {abs(mgr):.1f} percentage points'

txt3 = ('The FRBNY data show mortgage debt, including home '+
        f'equity lines of credit, totals \${tot["Mortgage Total"] / 1_000:,.2f} '+
        f'trillion in {d["qtr2"]}, equivalent to {shr["Mortgage Total"]:.1f} '+
        f'percent of disposable personal income. Student loans '+
        f'total \${tot["Student Loan"]:,.0f} billion, or '+
        f'{shr["Student Loan"]:.1f} percent of income; auto loans total '+
        f'\${tot["Auto Loan"]:,.0f} billion ({shr["Auto Loan"]:.1f} percent '+
        f'of income); and credit card debt is \${tot["Credit Card"]:,.0f} '+
        f'billion ({shr["Credit Card"]:.1f} percent of income).' )
write_txt(text_dir / 'hhcdebt4.txt', txt3)
print(txt3, '\n')
txt4 = ('Over the past three years, the ratio of total mortgage debt to '+
        f'disposable personal income {gr["Mortgage Total"]}, compared to '+
        f'{gr["Student Loan"]} for student loans, '+
        f'{gr["Auto Loan"]} for auto loans, and '+
        f'{gr["Credit Card"]} for credit card debt.')
write_txt(text_dir / 'hhcdebt5.txt', txt4)
print(txt4)

The FRBNY data show mortgage debt, including home equity lines of credit, totals \$11.99 trillion in the third quarter of 2022, equivalent to 64.1 percent of disposable personal income. Student loans total \$1,574 billion, or 8.4 percent of income; auto loans total \$1,524 billion (8.2 percent of income); and credit card debt is \$925 billion (4.9 percent of income). 

Over the past three years, the ratio of total mortgage debt to disposable personal income grew by 4.4 percentage points, compared to a decrease of 0.7 percentage point for student loans, an increase of 0.2 percentage point for auto loans, and a decrease of 0.4 percentage point for credit card debt.


In [35]:
# Household debt and consumer credit table
dtlt = pd.to_datetime(data.index[-1])
dt2 = pd.to_datetime(data.index[-2])
dt3y = pd.to_datetime(data.index[-13])
dt13 = pd.to_datetime('2013-01-01')
dt03 = pd.to_datetime('2003-01-01')

dts = [dtlt, dt2]
dts2 = [dtlt, dt2, dt3y, dt13, dt03]

for x in [data, sh]:
    x['Mortgage Total'] = x['Mortgage'] + x['HE Revolving']
    x['Non-Mortgage Total'] = (x['Auto Loan'] + x['Credit Card'] 
                               + x['Student Loan'] + x['Other'])

# Attempt to handle CCP coming out first
for x in [table_store_fa, table_store_fa_dpi]:
    if dtlt not in x.index:
        x.loc[dtlt, :] = '--'

d1 = {'Total': 'Financial Accounts Total',
      'Mortgages': '\hspace{2mm} \cbox{blue!60!violet} Mortgage Debt Total',
      'Consumer Credit': '\hspace{2mm} \cbox{magenta} Consumer Credit',
      'Other': '\hspace{2mm} \cbox{orange!80!yellow} Other'}

d2 = {'Total': 'Consumer Credit Panel Total',
      'Mortgage Total': '\hspace{2mm} Mortgage Debt Total',
      'Mortgage': '\hspace{4mm} Mortgage',
      'HE Revolving': '\hspace{4mm} Home Equity Revolving',
      'Non-Mortgage Total': '\hspace{2mm} Consumer Credit',
      'Auto Loan': '\hspace{4mm} \cbox{blue!60!cyan} Auto Loan',
      'Credit Card': '\hspace{4mm} \cbox{red} Credit Card',
      'Student Loan': '\hspace{4mm} \cbox{green!80!blue} Student Loan',
      'Other': '\hspace{4mm} Other'}

final = pd.DataFrame()

for dt in dts:
    dtmp = f'{dt.year} Q{dt.quarter}'
    for srs in [table_store_fa]:
        for k, v in d1.items():
            if srs.loc[dt, k] != '--':
                final.at[v, dtmp] = f'\${srs.loc[dt, k]:.2f}T'
            else: final.at[v, dtmp] = srs.loc[dt, k]
    for srs in [data]:
        for k, v in d2.items():            
            final.at[v, dtmp] = f'\${srs.loc[dt, k]:.2f}T' 
            
for dt in dts2:
    dtmp = f'`{str(dt.year)[2:]} Q{dt.quarter}'
    for srs in [table_store_fa_dpi]:
        for k, v in d1.items():
            if srs.loc[dt, k] != '--':
                final.at[v, dtmp] = round(srs.loc[dt, k], 1)
            else: 
                final.at[v, dtmp] = srs.loc[dt, k]
    for srs in [sh]:
        for k, v in d2.items():
            final.at[v, dtmp] = round(srs.loc[dt, k], 1)
            
final.to_csv(data_dir / 'hhcdebt.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

### Income as return on total HH assets

In [36]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=Z1&series=906ccd6e7fcae1e4f20ac00b86ade272&lastobs=&'
dt = 'from=03/01/1988&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d = {'LM152010005.Q': 'Nonfinancial',
     'FL154090005.Q': 'Financial'}

r = requests.get(url)
df = pd.read_csv(io.StringIO(r.content.decode('utf-8')), skiprows=5, 
                 index_col=0, parse_dates=True)[d.keys()].rename(d, axis=1)
df['Total'] = df['Nonfinancial'] + df['Financial']
df['DPI'] = nipa_df(retrieve_table('T20100')['Data'], ['A067RC'])
df['DPINF'] = df['Nonfinancial'].divide(df['Total'], axis=0)
df['DPIF'] = df['Financial'].divide(df['Total'], axis=0)
df['DPIsh'] = df['DPI'].divide(df['Total'], axis=0)
df['DPINFsh'] = df['DPINF'] * df['DPIsh']
df['DPIFsh'] = df['DPIF'] * df['DPIsh']

df = df * 100
df.loc['1989':,'DPIsh'].to_csv(data_dir / 'dpish.csv', index_label='date')

color = 'red'
node = end_node(df['DPIsh'], color=color, percent=True, date='q', 
                offset=0.35, full_year=True)
write_txt(text_dir / 'dpish_node.txt', node)

ltdt = dtxt(df.index[-1])['qtr1']
tot = df.DPIsh.iloc[-1]
tot90s = df.loc['1990':'1999', 'DPIsh'].mean()

text = (f'In {ltdt}, disposable income is equivalent to {tot:.1f} '+
        f'percent of the market value of US assets {c_line(color)}, '+
        f'compared to an average of {tot90s:.1f} percent during '+
        'the 1990s.')
write_txt(text_dir / 'dpishta.txt', text)
print(text)

In 2022 Q2, disposable income is equivalent to 11.3 percent of the market value of US assets (see {\color{red}\textbf{---}}), compared to an average of 16.0 percent during the 1990s.


### Household assets

See B.101, for example

In [37]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=Z1&series=5f48b7338e558e73e11dc78be7354a87&lastobs=&'+ 
       'from=03/01/1988&to=12/31/2022&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, clean_data = clean_fed_data(url, dtype='full')

sel = {'FA156012005.Q': ('DPI', 'Disposable personal income'),
       'FL152000005.Q': ('TOT', 'Total assets'),
       'LM152010005.Q': ('NFA', 'Non-financial assets'),
       'LM155035015.Q': ('HRE', 'Owner-occupied real estate'),
       'FL155035065.Q': ('REQ', 'Owner equity in real estate'),
       'LM155111005.Q': ('CDG', 'Consumer durable goods'),
       'FL154090005.Q': ('TFA', 'Financial assets'),
       'FL154000025.Q': ('DEP', 'Deposits'),
       'LM153064475.Q': ('CEQ', 'Corporate equities'),
       'LM152090205.Q': ('NEQ', 'Noncorporate equity'),
       'LM155035005.Q': ('RET', 'Real estate')}

colors = {'TEQ': 'blue!65!black', 'HRE': 'green!60!teal', 
          'DEP': 'cyan!40!white', 'DSL': 'blue!55!cyan'}
cbox = {name: c_box(color) for name, color in colors.items()}

sel_cols = [k for k,v in sel.items()]
col_names = [v[0] for k,v in sel.items()]

df = clean_data.loc[:,sel_cols]
df.columns = col_names

df['NPA'] = clean_data.loc[:,['LM165013765.Q', 'LM165015205.Q', 'LM165035005.Q']].sum(axis=1)
df['DSL'] = clean_data.loc[:,['LM154022375.Q', 'FL154023005.Q']].sum(axis=1)
df['TEQ'] = df['NEQ'] + df['CEQ']
df['OFA'] = df['TFA'] - df['DEP'] - df['DSL'] - df['TEQ']
df['OTH'] = df['TOT'] - df['HRE'] - df['DEP'] - df['DSL'] - df['TEQ']

pce = nipa_df(retrieve_table('T20304')['Data'], ['DPCERG']).loc[df.index,'DPCERG']
pr = (pce / pce.iloc[-1])

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC']).loc[df.index,'A191RC']

dfgdp = df.div(gdp, axis=0).dropna() *100
dfgdp.loc['1989':].to_csv(data_dir / 'hhassetsgdp.csv', 
                          index_label='date', float_format='%g')

# Nodes for latest values
cols = ['HRE', 'DEP', 'DSL', 'TEQ']
sdf = dfgdp[cols].iloc[-1]
height = ((sdf.cumsum() - (sdf / 2) + 15)).to_dict()
val = sdf.to_dict()
dtp = dtxt(dfgdp.index[-1] + pd.DateOffset(months=2))['datetime']
nodes = [f'\\absnode{{{{{dtp}}}}}{{{height[i]}}}{{\scriptsize {val[i]:.1f}}}' 
         for i in cols]
dtv = dtxt(dfgdp.index[-1])['qtr1'].replace(' ', '\\\\ \scriptsize ')
dtn = f'\\absnode{{{{{dtp}}}}}{{{sdf.cumsum().iloc[-1] + 25.0}}}{{\scriptsize {dtv}:}}'
nodes.append(dtn)
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'hhasset_nodes.txt', nodetext)

# Text1 
ltdt = dtxt(df.index[-1])['qtr1']
prdt = dtxt(df.index[-5])['qtr1']
i = df.iloc[-1] / 1000000
vt = {n: f'\${v:.1f} trillion' for n, v in i.items()}
totgdp = dfgdp['TOT'].iloc[-1]
g = {n: f'{v:.0f} percent of GDP' for n, v in dfgdp.iloc[-1].items()}
s = (df.div(df.TOT, axis=0) * 100).iloc[-1]

text = ('According to the US Financial Accounts produced by the Federal Reserve, '+
        'the market value of \\textbf{household and nonprofit assets} is '+
        f'{vt["TOT"]} in {ltdt}, equivalent to {totgdp:.0f} percent--or '+
        f'{totgdp/100:.3g} years--of GDP. Of this, {vt["NFA"]}, or '+
        f'{s.NFA:.1f} percent of the total, are tangible (non-financial) assets '+
        f'and {vt["TFA"]}, or {s.TFA:.1f} percent, are financial assets.\n\n'+
        "Tangible assets include peoples' homes as well "+
        'as consumer durable goods, such as cars, furniture, and appliances. '+
        f'Owner-occupied real estate is valued at {vt["HRE"]} '+
        f'in {ltdt}, equivalent to {g["HRE"]} {cbox["HRE"]}. The replacement '+
        f'value of consumer durable goods is {vt["CDG"]}, or {g["CDG"]}.\n\n'+
        'Financial assets include equity in businesses--corporate and '+
        f'non-corporate--with a market value of {vt["TEQ"]}, or '+
        f'{g["TEQ"]} {cbox["TEQ"]}, in {ltdt}. Debt securities and loan assets '+
        f'total {vt["DSL"]}, or {g["DSL"]} {cbox["DSL"]}. Cash and deposits, '+
        f'including money market accounts, total {vt["DEP"]}, or '+
        f'{g["DEP"]} {cbox["DEP"]}. Other financial assets total {vt["OFA"]}.')

write_txt(text_dir / 'hhasset.txt', text)
print(text)

According to the US Financial Accounts produced by the Federal Reserve, the market value of \textbf{household and nonprofit assets} is \$162.7 trillion in 2022 Q2, equivalent to 644 percent--or 6.44 years--of GDP. Of this, \$54.0 trillion, or 33.2 percent of the total, are tangible (non-financial) assets and \$108.7 trillion, or 66.8 percent, are financial assets.

Tangible assets include peoples' homes as well as consumer durable goods, such as cars, furniture, and appliances. Owner-occupied real estate is valued at \$41.2 trillion in 2022 Q2, equivalent to 163 percent of GDP (see \cbox{green!60!teal}). The replacement value of consumer durable goods is \$7.7 trillion, or 30 percent of GDP.

Financial assets include equity in businesses--corporate and non-corporate--with a market value of \$54.7 trillion, or 217 percent of GDP (see \cbox{blue!65!black}), in 2022 Q2. Debt securities and loan assets total \$10.5 trillion, or 42 percent of GDP (see \cbox{blue!55!cyan}). Cash and deposits

In [38]:
tnames = {'HRE': 'the value of owner-occupied real estate', 
          'DEP': 'money held as deposits',
          'DSL': 'the market value of debt and loan securities', 
          'TEQ': 'the market value of business equity',
          'OTH': 'other assets'}

real_data = df.div(pr, axis=0)
growth = growth_contrib_ann(real_data, 'TOT')

(growth.dropna()[['HRE', 'DEP', 'DSL', 'TEQ', 'OTH']].loc['1989':]
 .to_csv(data_dir / 'hh_asset_growth.csv', index_label='date'))
gr = growth.iloc[-1]
gr2 = growth[tnames.keys()].iloc[-1].rename(tnames)
gctext, bbdb = gc_desc(gr2, 2, 5)
grtot = value_text(gr.TOT, casual=True)
text = (f'The value of household and nonprofit assets {grtot} '+
        f'over the year ending {ltdt}. The {gctext}')
write_txt(text_dir / 'hhasset2.txt', text)
print(text)

The value of household and nonprofit assets fell 4.5 percent over the year ending 2022 Q2. The decrease is driven largely by a decrease in the market value of business equity, and partially offset by an increase in the value of owner-occupied real estate.


In [39]:
# Generate table
nd = {'TOT': '&Total Assets', 'NFA': '& \hspace{1mm} Non-financial assets', 
      'HRE': '\cbox{green!60!teal!95!black} & \hspace{3mm} Owner-occupied real estate', 
      'CDG': ' & \hspace{3mm} Consumer durable goods',
      'NPA': ' & \hspace{3mm} Nonprofit assets',
      'TFA': ' & \hspace{1mm} Financial assets',
      'DEP': '\cbox{cyan!32!white} & \hspace{3mm} Deposits, incl. money market',
      'DSL': '\cbox{blue!58!cyan} & \hspace{3mm} Debt securities and loans',
      'TEQ': '\cbox{blue!55!black} & \hspace{3mm} Business equity',
      'CEQ': ' & \hspace{5mm} Corporate equities',
      'NEQ': ' & \hspace{5mm} Noncorporate business equity'}

table = pd.DataFrame()
table[ltdt] = pd.Series({idx: f'\${val:.1f}' if idx == 'TOT' else f'{val:.1f}' 
                          for idx, val in i.items()})
table[ltdt+'  '] = dfgdp.iloc[-1]
table[prdt+'  '] = dfgdp.iloc[-5]
table['One-year'] = real_data.pct_change(4).iloc[-1] * 100
table['Three-year'] = ((real_data.pct_change(12) + 1)**(1/3) - 1).iloc[-1] * 100
table['20-year'] = ((real_data.pct_change(80) + 1)**(1/20) - 1).iloc[-1] * 100

table.index.name = '& '
clean = table.loc[nd.keys()].rename(nd).round(1)

clean.to_csv(data_dir / 'hhasset.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

# Version for download
clean.index = [i[-1] for i in clean.index.str.split('}')]
clean.index = clean.index.str.replace('&', '')
clean.to_csv(data_dir / 'hhasset_table.csv', index_label='category')

### Net Worth

In [40]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=Z1&series=5274f1fc3a4900aba158b78578142b2a&lastobs=&'
dt = 'from=03/01/1988&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)
n = {'FL152090005.Q': 'NW',
     'FL152000005.Q': 'ASSETS',
     'FL154190005.Q': 'LIAB',
     'FA156012005.Q': 'DPI'}

df = clean_data.rename(n, axis=1) / 1_000_000
datelt = dtxt(df.index[-1])['qtr1']
i = df.iloc[-1]
text = (f'In {datelt}, household and nonprofit institution net worth '+
        f'was \${i.NW:.1f} trillion, equivalent to {i.NW / i.DPI:.1f} '+
        'years of disposable personal income; the result of total '+
        f'assets of \${i.ASSETS:.1f} trillion and total liabilities '+
        f'of \${i.LIAB:.1f} trillion.')
write_txt(text_dir / 'nw1.txt', text)
print(text, '\n\n')

pce = (nipa_df(retrieve_table('T20304')['Data'], ['DPCERG'])
       .loc[df.index, 'DPCERG'])
res = (df.divide(pce / pce.iloc[-1], axis=0)
         .pct_change(4) * 100).dropna()
res.to_csv(data_dir / 'rdpi_nw.csv', index_label='date')

nwtxt = value_text(res.NW.iloc[-1], style='increase_by')
dpitxt = value_text(res.DPI.iloc[-1], style='increase_by')
i = res.iloc[-13:].mean()
dpi3txt = value_text(i.DPI, adj='average', casual=True)
nw3txt = value_text(i.NW, adj='average', casual=True)
text = (f'In {datelt}, inflation-adjusted net worth {nwtxt}'+
        ' (see\cbox{cyan!40!white}), and '+
        f'inflation-adjusted after-tax income {dpitxt} '+
        '(see {\color{blue!50!violet}\\textbf{---}}). Over '+
        f'the past three years, real net worth {nw3txt}, '+
        f'while real after-tax income {dpi3txt}')
write_txt(text_dir / 'nw2.txt', text)
print(text)

In 2022 Q2, household and nonprofit institution net worth was \$143.8 trillion, equivalent to 7.8 years of disposable personal income; the result of total assets of \$162.7 trillion and total liabilities of \$18.9 trillion. 


In 2022 Q2, inflation-adjusted net worth decreased by 5.1 percent (see\cbox{cyan!40!white}), and inflation-adjusted after-tax income decreased by 4.4 percent (see {\color{blue!50!violet}\textbf{---}}). Over the past three years, real net worth grew at an average rate of 7.4 percent, while real after-tax income grew at an average rate of 1.8 percent


### Net worth contributions

In [41]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=Z1&series=73021951e1b749df8a5de36975a7926d&lastobs=&'
dt = 'from=03/01/1988&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

df = clean_data.loc[:, ['FC152090005.Q', 'FU155060005.Q', 'FV158090005.Q', 
                 'FU156012005.Q', 'FR158000005.Q']]
df.columns = ['NW', 'NI', 'OVC', 'DPI', 'RV']

df = df.rolling(4).sum().dropna()
rate = (df['NI'] / df['DPI']).mean()
rate2 = (df['NI'] / df['DPI']).iloc[-1]

df['INC'] = df['DPI'] * rate
df['INV'] = df['NI'] - df['INC']
df['NWL'] = clean_data['FL152090005.Q']

growth = (df[['OVC', 'INC', 'INV', 'RV']]
          .div(df['NWL'].shift(4), axis=0).dropna() * 100)

growth.to_csv(data_dir / 'nw_gr.csv', index_label='date')

# Text for chart
ldate = dtxt(growth.index[-1])['qtr1']
ltdate = dtxt(growth.index[-1])['qtr2']

hgtxt = value_text(growth['RV'].iloc[-1], style='contribution_to', 
           ptype='pp', threshold=0.1)
inctxt = value_text(growth['INC'].iloc[-1], style='contribution', 
           ptype='pp', threshold=0.1)
inv = growth['INV'].iloc[-1]
invtxt = [f'; an additional {inv:.1f} percentage points were added' if inv >= 0.1 
         else f', but {abs(inv):.1f} percentage points were subtracted' if inv <= -0.1 
         else ', and cyclical activity in investment did not seem to play a role'][0]
othtxt = value_text(growth['OVC'].iloc[-1], style='contribution', 
           ptype='pp', threshold=0.1)
hg3txt = value_text(growth['RV'].iloc[-13:].mean(), style='contribution', 
           ptype='pp', threshold=0.1)
oth3txt = value_text(growth['OVC'].iloc[-13:].mean(), style='contribution', 
           ptype='pp', threshold=0.1)
ni3 = growth['INC'].iloc[-13:].mean() + growth['INV'].iloc[-13:].mean()
ni3txt = value_text(ni3, style='contribution', 
           ptype='pp', threshold=0.1)

text = (f'In the {ltdate}, holding gains {hgtxt} the {growth.iloc[-1].sum():.1f} '+
        'percent change in net worth. Income invested at the 1989-onward average '+
        f'rate of {rate*100:.1f} percent would have {inctxt}{invtxt} as household '+
        f'net investment was {rate2*100:.1f} percent of disposable person income '+
        f'in {ldate}. Other volume changes {othtxt}.\n\nOver the past three years, '+
        f'net worth grew at an average rate of {growth.iloc[-13:].mean().sum():.1f} '+
        f'percent. Holding gains {hg3txt} to this total, on average; net investment '+
        f'of income {ni3txt}; and other volume changes {oth3txt}.')
write_txt(text_dir / 'nwcontrib.txt', text)
print(text)

In the the second quarter of 2022, holding gains subtracted 0.5 percentage point from the 1.1 percent change in net worth. Income invested at the 1989-onward average rate of 10.2 percent would have contributed 1.3 percentage points, but 0.3 percentage points were subtracted as household net investment was 7.9 percent of disposable person income in 2022 Q2. Other volume changes contributed 0.6 percentage point.

Over the past three years, net worth grew at an average rate of 10.6 percent. Holding gains contributed 8.1 percentage points to this total, on average; net investment of income contributed 2.2 percentage points; and other volume changes contributed 0.3 percentage point.


### Equity Payout

In [42]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=Z1&series=5dbfee986a7636f1bc997a80c313cabc&lastobs=&'+
       'from=01/01/1988&to=12/31/2022&filetype=csv&'+
       'label=include&layout=seriescolumn')
d, df = clean_fed_data(url)
srs = {'FA103164103.Q': 'Buybacks',
       'FA106121075.Q': 'Dividends'}

data = df.rename(srs, axis=1)
data['Buybacks'] = -data['Buybacks']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
res = data.divide(gdp, axis=0).loc['1989':].dropna() * 100
res.to_csv(data_dir / 'eq_payout.csv', index_label='date')

In [43]:
cbd = c_box('blue!90!purple')
cbb = c_box('magenta')

ltdt = dtxt(res.index[-1])['qtr2']
ltd = res['Dividends'].iloc[-1]
ltb = res['Buybacks'].iloc[-1]
d19 = res.loc['2019', 'Dividends'].mean()
b19 = res.loc['2019', 'Buybacks'].mean()
avd = res.loc['1990': '2015', 'Dividends'].mean()
avb = res.loc['1990': '2015', 'Buybacks'].mean()
text = (f'In {ltdt}, nonfinancial corporation net dividends are '+
        f'equivalent to {ltd:.1f} percent of GDP {cbd} and net '+
        f'equities issuance is equivalent to {ltb:.1f} percent of '+
        f'GDP {cbb}. In 2019, net dividends were {d19:.1f} percent '+
        f'of GDP and net issuance was {b19:.1f} percent. From 1990 to '+
        f'2015, net dividends averaged {avd:.1f} percent of GDP and '+
        f'net issuance averaged {avb:.1f} percent.')
write_txt(text_dir / 'eq_payout.txt', text)
print(text)

In the second quarter of 2022, nonfinancial corporation net dividends are equivalent to 2.8 percent of GDP (see \cbox{blue!90!purple}) and net equities issuance is equivalent to 2.1 percent of GDP (see \cbox{magenta}). In 2019, net dividends were 3.3 percent of GDP and net issuance was 2.1 percent. From 1990 to 2015, net dividends averaged 2.6 percent of GDP and net issuance averaged 1.3 percent.


### Government Share of US Net Worth

In [44]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=Z1&series=94066b162cbbe774fa6cb1ae8cf757e6&lastobs=&'+
       'from=01/01/1988&to=12/31/2022&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, df = clean_fed_data(url)

s = {'FL212090095.Q': 'SLG NW',
     'FL312090095.Q': 'FG NW',
     'FL892090005.Q': 'Total NW'}
data = df.rename(s, axis=1)
data['Gov NW'] = data['SLG NW'] + data['FG NW']
res = ((data.div(data['Total NW'], axis=0) * 100)
            .drop('Total NW', axis=1).loc['1989':])
res.to_csv(data_dir / 'govshnw.csv', index_label='date', 
           float_format='%g')

In [45]:
colors = {'Gov NW': 'green!80!blue', 'SLG NW': 'blue!90!black', 
          'FG NW': 'orange!80!white'}
date = {k: None for k, v in colors.items()}
date[res.iloc[-1].idxmax()] = 'q'
nodes = '\n'.join([end_node(res[k], v, percent=True, date=date[k], 
                            full_year=True) 
                   for k, v in colors.items()])
write_txt(text_dir / 'govnw_nodes.txt', nodes)

totval = data['FG NW'].iloc[-1] / 1_000_000
ns = '' if totval > 0 else '-'
totv = f'{ns}\${abs(totval):.1f} trillion'
ltdt = dtxt(data.index[-1])['qtr1']
gsh = f"{res['Gov NW'].iloc[-1]:.1f} percent"
fgsh = f"{res['FG NW'].iloc[-1]:.1f} percent"
slgsh = f"{res['SLG NW'].iloc[-1]:.1f} percent"
cl = {k: c_line(v) for k, v in colors.items()}
text = ('The combined US government sector has a net worth of '+
        f'{totv}, as of {ltdt}, equivalent to {gsh} of '+
        f'national wealth {cl["Gov NW"]}. Federal government '+
        'net worth (excluding land) is equal to '+
        f'{fgsh} of national wealth {cl["FG NW"]}, while '+
        'state and local government net worth is equivalent to '+
        f'{slgsh} {cl["SLG NW"]}.')
write_txt(text_dir / 'govshnw.txt', text)
print(text)

The combined US government sector has a net worth of -\$20.6 trillion, as of 2022 Q2, equivalent to -7.8 percent of national wealth (see {\color{green!80!blue}\textbf{---}}). Federal government net worth (excluding land) is equal to -15.8 percent of national wealth (see {\color{orange!80!white}\textbf{---}}), while state and local government net worth is equivalent to 8.0 percent (see {\color{blue!90!black}\textbf{---}}).


### Government Assets

In [46]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=Z1&series=69790f5ee559de8e4c534f51c91cc3bb&lastobs=&'+
       'from=01/01/1988&to=12/31/2022&'+
       'filetype=csv&label=include&layout=seriescolumn')
d, data = clean_fed_data(url)
s = {'FL212010095.Q': 'SLG NFA',
     'FL214090005.Q': 'SLG FA',
     'FL312010095.Q': 'FG NFA',
     'FL314090005.Q': 'FG FA'}
df = data.rename(s, axis=1)

# Share of GDP
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
sh = (df.divide(gdp['A191RC'], axis=0) * 100).loc['1989':].dropna()
sh.to_csv(data_dir / 'govassets.csv', index_label='date', 
          float_format='%g')

ltdt = dtxt(sh.index[-1])
totval = df.iloc[-1].sum() / 1_000_000
sh['Tot'] = sh.sum(axis=1)
lt = sh.iloc[-1].apply(lambda x: f'{x:.1f} percent of GDP')


colors = {'SLG NFA': 'cyan!36!white', 
          'SLG FA': 'blue!75!cyan', 'FG NFA': 'violet!60!white',
          'FG FA': 'blue!28!violet!90!black'}
cl = {key: c_box(value) for key, value in colors.items()}

text = (f'In {ltdt["qtr2"]}, the market value of government assets, '+
        f'excluding land, is \${totval:.1f} trillion, equivalent '+
        f'to {lt["Tot"]}. Of this, state and local government '+
        f'nonfinancial assets, such as buildings and equipment, '+
        f'are equivalent to {lt["SLG NFA"]} {cl["SLG NFA"]}, and '+
        'state and local government financial assets, such as '+
        f'insurance trust funds, are equivalent to {lt["SLG FA"]} '+
        f'{cl["SLG FA"]}.\n\nThe market value of federal '+
        'government nonfinancial assets is equivalent to '+
        f'{lt["FG NFA"]} in {ltdt["qtr1"]} {cl["FG NFA"]}. '+
        'Federal government financial assets are valued at '+
        f'{lt["FG FA"]} {cl["FG FA"]}.')
write_txt(text_dir / 'govassets.txt', text)
print(text)

In the second quarter of 2022, the market value of government assets, excluding land, is \$27.6 trillion, equivalent to 109.2 percent of GDP. Of this, state and local government nonfinancial assets, such as buildings and equipment, are equivalent to 59.1 percent of GDP (see \cbox{cyan!36!white}), and state and local government financial assets, such as insurance trust funds, are equivalent to 17.4 percent of GDP (see \cbox{blue!75!cyan}).

The market value of federal government nonfinancial assets is equivalent to 16.8 percent of GDP in 2022 Q2 (see \cbox{violet!60!white}). Federal government financial assets are valued at 15.8 percent of GDP (see \cbox{blue!28!violet!90!black}).


### Net Fixed Investment

In [47]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=Z1&series=fe4e292249867fb6016a0c9bd55e400d&'+
       'lastobs=&from=03/01/1988&to=12/31/2022&filetype=csv&'+
       'label=include&layout=seriescolumn')
d, clean_data = clean_fed_data(url)
clean_data.to_csv(data_dir / 'z1_nfi_raw.csv', index_label='date')

In [48]:
df = pd.read_csv(data_dir / 'z1_nfi_raw.csv', index_col='date',
                parse_dates=True)
gdp = nipa_df(retrieve_table('T10105')['Data'], 
              ['A191RC'])['A191RC']

d2 = {'FA115015005.Q': 'NCB',
      'FA155015005.Q': 'HH',
      'FA215015005.Q': 'SLG',
      'FA315015005.Q': 'FEDG',
      'FA885015005.Q': 'TOT',
      'FA885019005.Q': 'TOTG',
      'FA886300095.Q': 'CFC',
      'FA155111005.Q': 'DG',
      'FA105015085.Q': 'CB'}
res = df[d2.keys()].rename(d2, axis=1)
res['GOV'] = df[['FA215015005.Q', 'FA315015005.Q']].sum(axis=1)
res['BUS'] = res['TOT'] - res[['GOV', 'HH']].sum(axis=1)
final = (res.divide(gdp, axis=0).dropna() * 100).loc['1989':]
final.to_csv(data_dir / 'z1_nfi.csv', 
             index_label='date', float_format='%g')

In [49]:
dgcol = 'blue!80!green!95!white'
totcol = 'purple'
nodes = (end_node(final.TOT, totcol, date='q', 
                  percent=True, offset=0.35) + 
         '\n' + end_node(final.DG, dgcol, percent=True))
write_txt(text_dir / 'z1_nfi_nodes.txt', nodes)

In [50]:
ltr = res.iloc[-1] / 1_000_000
lts = final.iloc[-1]
s19 = final.loc['2019'].mean()
ltdt = dtxt(final.index[-1])['qtr1']
text = (f'In {ltdt}, gross fixed investment was \${ltr.TOTG:.1f} '+
        f'trillion, depreciation was \${ltr.CFC:.1f} trillion, and '+
        f'net fixed investment was \${ltr.TOT:.1f} trillion, '+
        f'equivalent to {lts.TOT:.1f} percent of GDP {c_line(totcol)}. '+
        f'In 2019, net fixed investment was {s19.TOT:.1f} percent of '+
        'GDP.\n\nSome economic measures also count consumer '+
        'durable goods, such as autos, furniture, and '+
        'appliances, as investments. Net investment in consumer '+
        f'durables was \${ltr.DG *1000:,.0f} billion in {ltdt}, or '+
        f'{lts.DG:.1f} percent of GDP {c_line(dgcol)}. Consumer '+
        f'durable goods net fixed investment was {s19.DG:.1f} percent '+
        'of GDP in 2019.')
write_txt(text_dir / 'z1_nfi_main.txt', text)
print(text, '\n')
hh03 = final.loc['2003': '2006', 'HH'].mean()
hhcol = 'cyan!40!white!95!white'
buscol = 'magenta!90!white'
govcol = 'violet!85!black'
text = ('Levels of net fixed investment vary by sector and over time. '+
        f'In {ltdt}, household sector net fixed investment, excluding '+
        f'consumer durables, was equivalent to {lts.HH:.1f} percent '+
        f'of GDP, compared to {s19.HH:.1f} percent in 2019 '+
        f'{c_box(hhcol)}. From 2003 to 2006, during the housing bubble, '+
        f'household net fixed investment averaged {hh03:.1f} percent '+
        'of GDP. Business sector net fixed investment is equivalent to '+
        f'{lts.BUS:.1f} percent of GDP in {ltdt}, and {s19.BUS:.1f} '+
        f'percent in 2019 {c_box(buscol)}. Government net fixed '+
        f'investment is equivalent to {lts.GOV:.1f} percent of GDP in '+
        f'{ltdt} and {s19.GOV:.1f} percent in 2019 '+
        f'{c_box(govcol)}.')
write_txt(text_dir / 'z1_nfi_sect.txt', text)
print(text)

In 2022 Q2, gross fixed investment was \$5.3 trillion, depreciation was \$4.3 trillion, and net fixed investment was \$1.1 trillion, equivalent to 4.3 percent of GDP (see {\color{purple}\textbf{---}}). In 2019, net fixed investment was 4.9 percent of GDP.

Some economic measures also count consumer durable goods, such as autos, furniture, and appliances, as investments. Net investment in consumer durables was \$479 billion in 2022 Q2, or 1.9 percent of GDP (see {\color{blue!80!green!95!white}\textbf{---}}). Consumer durable goods net fixed investment was 1.2 percent of GDP in 2019. 

Levels of net fixed investment vary by sector and over time. In 2022 Q2, household sector net fixed investment, excluding consumer durables, was equivalent to 1.5 percent of GDP, compared to 1.3 percent in 2019 (see \cbox{cyan!40!white!95!white}). From 2003 to 2006, during the housing bubble, household net fixed investment averaged 3.5 percent of GDP. Business sector net fixed investment is equivalent to 2

In [51]:
d2 = {'TOT': '\hspace{0.1mm} {\color{purple}\\textbf{---}} Net Fixed Investment',
      'BUS': '\hspace{0.5mm}\cbox{magenta!90!white}Business',
      'NCB': '\hspace{6mm} Nonfin. Noncorp. Business',
      'CB': '\hspace{6mm} Nonfin. Corporations',      
      'GOV': '\hspace{0.5mm}\cbox{violet!85!black}Government',
      'SLG': '\hspace{6mm} State \& Local Gov.',
      'FEDG': '\hspace{6mm} Federal Gov.',
      'HH': '\hspace{0.5mm}\cbox{cyan!40!white!95!white}Household \& Nonprofit',
      'DG': '\hspace{0.1mm} {\color{blue!80!green!95!white}\\textbf{---}} /\cbox{blue!80!green!95!white}Consumer Durables'}

fin = final[list(d2.keys())].rename(d2, axis=1)
tbl = fin.iloc[-3:].iloc[::-1].T
tbl[fin.index[-5]] = fin.iloc[-5]
cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(tbl.columns)]

tbl.columns = cols
tbl['2019'] = fin.loc['2019'].mean()
tbl['2003 --`06'] = fin.loc['2003':'2006'].mean()
tbl['1999 --`01'] = fin.loc['1999':'2001'].mean()
tbl = tbl.applymap('{:.2f}'.format)
tbl.to_csv(data_dir / 'z1_nfi.tex', sep='&', lineterminator='\\\ ', 
            quotechar=' ')

### Rest of World - Financial Account Balance

In [52]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=Z1&series=190320d1c0034f8438756dbb830ed2d2&lastobs=&'+
       'from=03/01/1988&to=12/31/2022&filetype=csv&label=include&'+
       'layout=seriescolumn')
s, df = clean_fed_data(url)
d = {'FA263181125.Q': 'Equity', 'FA264190005.Q': 'Liab',
     'FA264194005.Q': 'EqLiab', 'FA266000105.Q': 'Saving',
     'FA264090005.Q': 'Assets'}
df = df.rename(d, axis=1)
df['Discrepancy'] = (df['Saving'] - (df['Assets'] - df['EqLiab']))
for i in ['Equity', 'Liab', 'EqLiab']:
    df[i] = -df[i]
    
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
res = (df.divide(gdp['A191RC'], axis=0) * 100).rolling(4).mean().dropna()
res.loc['1989':].to_csv(data_dir/'fabz1.csv', index_label='date')

In [53]:
ltdt = dtxt(res.index[-1])['qtr2']
lta = df['Assets'].iloc[-1] / 1_000
lta = f'\${(lta / 1_000):.2f} trillion' if lta > 1_000 else f'\${(lta):.0f} billion'
sha = res['Assets'].iloc[-1]
shl = res['Liab'].iloc[-1]
she = res['Equity'].iloc[-1]

colors = {'Assets': 'blue!60!cyan!90!black', 
          'Liab': 'red', 'Equity': 'violet!90!black'}
cl = {key: c_box(value) for key, value in colors.items()}

text = (f'In {ltdt}, the rest of the world acquired '+
        f'{lta} in US assets, equivalent to {sha:.1f} percent '+
        f'of GDP {cl["Assets"]}. The rest of the world incurred the '+
        f'equivalent of {shl:.1f} percent of US GDP in liabilities '+
        f'{cl["Liab"]} and issued {she:.1f} percent of US GDP of '+
        f'equity in foreign businesses {cl["Equity"]}. ')
write_txt(text_dir / 'fabz1.txt', text)
print(text)

In the second quarter of 2022, the rest of the world acquired \$2.15 trillion in US assets, equivalent to 7.9 percent of GDP (see \cbox{blue!60!cyan!90!black}). The rest of the world incurred the equivalent of -2.7 percent of US GDP in liabilities (see \cbox{red}) and issued -2.7 percent of US GDP of equity in foreign businesses (see \cbox{violet!90!black}). 


### Distributive Financial Accounts (DFA)

https://www.federalreserve.gov/releases/z1/dataviz/download/dfa-networth-shares.csv

In [54]:
url = ('https://www.federalreserve.gov/releases/z1/'+
       'dataviz/download/dfa-networth-shares.csv')
r = requests.get(url)
df = pd.read_csv(io.StringIO(r.content.decode('utf-8')), 
                 index_col=['Date', 'Category'])['Net worth'].unstack()
df.index = pd.to_datetime(df.index.str.replace(':', '-'))
df['Top10'] = df[['RemainingTop1', 'Next9', 'TopPt1']].sum(axis=1)
df['Top1'] = df[['RemainingTop1', 'TopPt1']].sum(axis=1)
df.to_csv(data_dir / 'dfa_nw_sh.csv', index_label='date', float_format='%g')

In [55]:
df = pd.read_csv(data_dir / 'dfa_nw_sh.csv', index_col='date', 
                 parse_dates=True)
cols = [('Bottom50', 'blue!80!cyan'), ('Next40', 'cyan!80!white'), 
        ('Next9', 'orange!50!yellow'), ('Top1', 'red!50!orange')]

nodes = '\n'.join([end_node(df[name], col, percent=True) 
                 for name, col in cols])
write_txt(text_dir / 'dfa_nw_sh_nodes.txt', nodes)

lt = df.iloc[-1].apply(lambda x: f'{x:.1f} percent')
ltdt = dtxt(lt.name)['qtr1']
cl = {name: c_line(col) for name, col in cols}
ch = ((df.iloc[-1] - df.loc['1989'].mean())
      .apply(lambda x: value_text(x, ptype='pp')))

url = ('https://www.federalreserve.gov/releases/'+
       'z1/dataviz/dfa/distribute/chart/')
text = (f'The Federal Reserve \href{{{url}}}{{report}} net '+
        'worth by percentile. The top one percent of households '+
        f'by wealth own {lt.Top1} '+
        f'percent of US wealth, as of {ltdt} {cl["Top1"]}, while '+
        f'the top 10 percent of households own {lt.Top10} percent. '+
        f'The bottom half of households own {lt.Bottom50} percent of '+
        f'US wealth {cl["Bottom50"]}.\n\nSince 1989, the wealth '+
        f'share of the top one percent {ch.Top1}, while the share '+
        f'held by the bottom 50 percent {ch.Bottom50}. The wealth '+
        'share of the 40 percent of households in wealth percentiles '+
        f'50 through 90 {ch.Next40} since 1989.')
write_txt(text_dir / 'dfa_nw_sh.txt', text)
print(text)

The Federal Reserve \href{https://www.federalreserve.gov/releases/z1/dataviz/dfa/distribute/chart/}{report} net worth by percentile. The top one percent of households by wealth own 31.1 percent percent of US wealth, as of 2022 Q2 (see {\color{red!50!orange}\textbf{---}}), while the top 10 percent of households own 68.0 percent percent. The bottom half of households own 3.2 percent percent of US wealth (see {\color{blue!80!cyan}\textbf{---}}).

Since 1989, the wealth share of the top one percent increased 8.5 percentage points, while the share held by the bottom 50 percent decreased 0.5 percentage point. The wealth share of the 40 percent of households in wealth percentiles 50 through 90 decreased 7.5 percentage points since 1989.


# Survey of Consumer Finances

In [56]:
df = (pd.read_csv('../data/scf_wealth_median.csv')
        .set_index(['year', 'Category'])['Before_Tax_Income']
        .unstack())
df.index = pd.to_datetime([f'{year}-07-01' for year in df.index])
df.to_csv(data_dir / 'scfwealthmedian.csv', index_label='date', float_format='%g')
node = end_node(df['Less than 25'], 'violet')
write_txt(text_dir / 'scfwealthmedian_node1.txt', node)
node = end_node(df['25-49.9'], 'red!20!orange!95!black')
write_txt(text_dir / 'scfwealthmedian_node2.txt', node)
node = end_node(df['50-74.9'], 'green!80!blue!85!black')
write_txt(text_dir / 'scfwealthmedian_node3.txt', node)
node = end_node(df['75-89.9'], 'red!80!black')
write_txt(text_dir / 'scfwealthmedian_node4.txt', node)
node = end_node(df['90-100'], 'blue!80!cyan!80!black', 
          date='y', full_year=True)
write_txt(text_dir / 'scfwealthmedian_node5.txt', node)
#df.plot();

In [57]:
yr = dtxt(df.index[-1])['year']
pryr = dtxt(df.index[0])['year']

g1val = df['90-100'].iloc[-1] * 1000
g1valpr = df['90-100'].iloc[0] * 1000
g1ch = g1val - g1valpr
g1pc = ((g1val / g1valpr) - 1) * 100

g2val = df['50-74.9'].iloc[-1] * 1000
g2valpr = df['50-74.9'].iloc[0] * 1000
g2ch = g2val - g2valpr
g2pc = ((g2val / g2valpr) - 1) * 100

g3val = df['25-49.9'].iloc[-1] * 1000
g3valpr = df['25-49.9'].iloc[0] * 1000
g3ch = g3val - g3valpr
g3pc = ((g3val / g3valpr) - 1) * 100

g4val = df['Less than 25'].iloc[-1] * 1000
g4valpr = df['Less than 25'].iloc[0] * 1000
g4ch = g4val - g4valpr
g4pc = ((g4val / g4valpr) - 1) * 100

text1 = (f'have a median annual income of \${g1val:,.0f} in {yr} and '+
         f'\${g1valpr:,.0f} in {pryr}, after adjusting for inflation. '+
         f'Median income for the group increased \${g1ch:,.0f}, or '+
         f'{g1pc:.1f} percent, over the 30-year period. ')

write_txt(text_dir / 'income_wealthg1.txt', text1)
print(text1)

text2 = ('In contrast, families in the third quartile of wealth (50th to 74.9th '+
         f'percentiles, mean wealth of \$700,000 in {yr}, see '+
         '{\color{green!80!blue!85!black}\\textbf{---}}) have a median income of '+
         f'\${g2val:,.0f} in {yr} and \${g2valpr:,.0f} in '+
         f'{pryr}, an increase of \${g2ch:,.0f} ({g2pc:.1f} percent). \n\n'+
         'Median income for families in the second quartile (25th to 49.9th '+
         f'percentiles, mean wealth of \$236,000 in {yr}, see '+
         '{\color{red!20!orange!95!black}\\textbf{---}}) increased '+
         f'\${g3ch:,.0f} ({g3pc:,.1f} percent) to '+
         f'\${g3val:,.0f} in {yr}, '+
         f'from \${g3valpr:,.0f} in {pryr}. \n\n For '+
         'the bottom quarter of families by wealth (see {\color{violet}\\textbf{---}}), '+
         f'median income increased \${g4ch:,.0f} or {g4pc:,.1f} '+
         f'percent to \${g4val:,.0f}, over the 30 years ending {yr}. '+
         f'The average wealth of the bottom quarter of families '+
         f'is negative in {yr} and the median wealth is virtually zero. ')
write_txt(text_dir / 'income_wealthg2.txt', text2)
print('\n', text2)

have a median annual income of \$236,203 in 2019 and \$155,693 in 1989, after adjusting for inflation. Median income for the group increased \$80,510, or 51.7 percent, over the 30-year period. 

 In contrast, families in the third quartile of wealth (50th to 74.9th percentiles, mean wealth of \$700,000 in 2019, see {\color{green!80!blue!85!black}\textbf{---}}) have a median income of \$70,250 in 2019 and \$62,277 in 1989, an increase of \$7,973 (12.8 percent). 

Median income for families in the second quartile (25th to 49.9th percentiles, mean wealth of \$236,000 in 2019, see {\color{red!20!orange!95!black}\textbf{---}}) increased \$3,239 (7.4 percent) to \$46,833 in 2019, from \$43,594 in 1989. 

 For the bottom quarter of families by wealth (see {\color{violet}\textbf{---}}), median income increased \$8,766 or 42.2 percent to \$29,525, over the 30 years ending 2019. The average wealth of the bottom quarter of families is negative in 2019 and the median wealth is virtually zero. 


In [58]:
d = {'Black, non-Hispanic': 'Black', 
     'White, non-Hispanic': 'White'}

df = pd.read_csv('../data/scf_race_mean.csv').set_index(['year', 'Category'])['Net_Worth'].unstack()
df = df.rename(d, axis=1)
df.index = pd.to_datetime([f'{year}-07-01' for year in df.index])
df = df.round(1)
df.to_csv(data_dir / 'scfracemean.csv', index_label='date')

In [59]:
df = pd.read_csv('../data/scf_race_mean.csv').set_index(['year', 'Category'])['Before_Tax_Income'].unstack()
df = df.rename(d, axis=1)
df.index = pd.to_datetime([f'{year}-07-01' for year in df.index])
df = df.round(1)
df.to_csv(data_dir / 'scfracemean1.csv', index_label='date')

In [60]:
df = pd.read_csv('../data/scf_race_mean.csv').set_index(['year', 'Category'])['Financial_Assets'].unstack()
df = df.rename(d, axis=1)
df.index = pd.to_datetime([f'{year}-07-01' for year in df.index])
df = df.round(1)
df.to_csv(data_dir / 'scfracemean0.csv', index_label='date')

In [61]:
df = pd.read_csv('../data/scf_race_mean.csv').set_index(['year', 'Category'])['Stock_Holdings'].unstack()
df = df.rename(d, axis=1)
df.index = pd.to_datetime([f'{year}-07-01' for year in df.index])
df = df.round(1)
df.to_csv(data_dir / 'scfracemean2.csv', index_label='date')

wval = df.White.iloc[-1] * 1000
bval = df.Black.iloc[-1] * 1000

df = pd.read_csv('../data/scf_race_have.csv').set_index(['year', 'Category'])['Stock_Holdings'].unstack()
df = df.rename(d, axis=1)
df.index = pd.to_datetime([f'{year}-07-01' for year in df.index])
df = df.round(1)
df.to_csv(data_dir / 'scfracehave.csv', index_label='date')

wp = df.White.iloc[-1]
bp = df.Black.iloc[-1]

yr = dtxt(df.index[-1])['year']

text = (f'In {yr}, among the {wp:.1f} percent of white families who own stocks, '+
        f'the average value of stock holdings is \${wval:,.0f}. The return on these '+
        'assets is a supplement to labor income and the assets themselves provide '+
        'cushion against unexpected expenses. Meanwhile, black families '+
        f'have relatively few financial assets; only {bp:.1f} percent of black '+
        f'families own stocks, with average stock holdings of \${bval:,.0f}.')
write_txt(text_dir / 'scfracestocks.txt', text)
print(text)

In 2019, among the 60.8 percent of white families who own stocks, the average value of stock holdings is \$433,900. The return on these assets is a supplement to labor income and the assets themselves provide cushion against unexpected expenses. Meanwhile, black families have relatively few financial assets; only 33.5 percent of black families own stocks, with average stock holdings of \$76,300.
